In [24]:
import requests

standings_2024_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [16]:
#Get HTML from server
data = requests.get(standings_2024_url)

In [ ]:
#Grab all href tags from <a> in the standings table, as we want data from all teams
from bs4 import BeautifulSoup

#Initialize soup object
soup = BeautifulSoup(data.text)
soup

In [18]:
#Select the table element using table.stats_table css selector, specifically select the first table
standings_table = soup.select('table.stats_table')[0]


IndexError: list index out of range

In [5]:
links = standings_table.find_all('a')
links = [l.get("href") for l in links]

#Want to remove all links to do with players and only have links to do with clubs
links = [l for l in links if "/squads" in l] #Include only links with "/squad" inside the link
links #Note that all of these urls are relative paths

NameError: name 'standings_table' is not defined

In [6]:
#Add full paths by concatenating string to all
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls


NameError: name 'links' is not defined

In [50]:
arsenal_url = team_urls[5]
#Get HTML from Arsenal page
arsenal_data = requests.get(arsenal_url)

# arsenal_fixtures = soup.select('table.stats_table')[0]




In [25]:
import pandas as pd

#Use HTML to read into pd df, pd will detect tables
#Match parameter - The set of tables containing text matching this regex or string will be returned.
matches = pd.read_html(arsenal_data.text, match="Scores & Fixtures")[0]

NameError: name 'arsenal_data' is not defined

# Extract shooting stats from each match

In [52]:
soup = BeautifulSoup(arsenal_data.text)
links = soup.find_all("a") #Get all the <a> tags, links at this point includes array full of a tags, we want to get the urls from these a tags (href from a)
links = [l.get("href") for l in links ] #Get all the hrefs
links = [l for l in links if l and 'all_comps/shooting' in l]
links


[]

In [53]:
shooting_html = requests.get(f"https://fbref.com{links[0]}")
shooting_table = pd.read_html(shooting_html.text, match="Shooting")[0]
shooting_table.head() #From head here we can see that there is multi-index level as there are two rows of columns, generally this is not necessary

#We want to be able to index by the column names and so we can drop level
shooting_table.columns = shooting_table.columns.droplevel()
shooting_table.columns

IndexError: list index out of range

In [54]:
shooting_cols = ["Date", "Sh", "SoT", "SoT%", "G/Sh", "G/SoT", "FK", "PK", "xG"]

match_shape = matches.shape
shooting_shape = shooting_table.shape
print(match_shape, shooting_shape) #(52,19) and (53,26), our dataset is not perfect and for some reason there is a match that is not logged in the table


#With the above in mind when we merge we will only combine the matching dates in both
team_data = matches.merge(shooting_table[shooting_cols], on="Date")
team_data.head()
# print(team_data.shape) #Shape is (52,27)

(51, 19) (52, 26)


,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,SoT%,G/Sh,G/SoT,FK,PK,xG_y
0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Home,D,1,1,Liverpool,...,Match Report,NaN,10,4,40.0,0.10,0.25,1.0,0,1.4
1,2023-08-20,16:30,Premier League,Matchweek 2,Sun,Away,L,1,3,West Ham,...,Match Report,NaN,16,3,18.8,0.06,0.33,0.0,0,2.5
2,2023-08-25,20:00,Premier League,Matchweek 3,Fri,Home,W,3,0,Luton Town,...,Match Report,NaN,19,8,42.1,0.16,0.38,1.0,0,2.2
3,2023-08-30,19:45,EFL Cup,Second round,Wed,Home,W,2,1,AFC Wimbledon,...,Match Report,NaN,23,9,39.1,0.04,0.11,NaN,1,NaN
4,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,L,0,1,Nott'ham Forest,...,Match Report,NaN,21,2,9.5,0.00,0.00,2.0,0,2.3


## Scrape data for multiple teams across multiple seasons

In [15]:
years = list(range(2024,2021, -1))
years

[2024, 2023, 2022]

In [22]:
  df = pd.read_html("https://fbref.com/en/comps/9/Premier-League-Stats")[0]
  print(df)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)>

In [16]:
all_matches = []

In [17]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [18]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
import random

user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
]


headers = {'User-Agent': random.choice(user_agents)}

shooting_cols = ["Date", "Sh", "SoT", "SoT%", "G/Sh", "G/SoT", "FK", "PK", "xG"]

for year in years: #Scrape each season
  # data = requests.get(standings_url, headers = headers)
  data = pd.read_html(standings_url)
  soup = BeautifulSoup(data.text)
  standings_table = soup.select('table.stats_table')[0]

  links = standings_table.find_all('a')
  links = [l.get("href") for l in links]
  links = [l for l in links if "/squads" in l]

  team_urls = [f"https://fbref.com{l}" for l in links]

  previous_season = soup.select("a.prev")[0].get("href")
  standings_url =f"https://fbref.com{previous_season}"

  for team_url in team_urls: #Scrape each team in a given season
    team_name = team_url.split("/")[-1].replace("-Stats","").replace("-","") #Get the team name
    team_html = requests.get(team_url)
    soup = BeautifulSoup(team_html.text)
    print(team_name)

    matches = pd.read_html(team_html.text, match="Scores & Fixtures")[0] #Get table of match data

    links = soup.find_all('a')
    links = [l.get("href") for l in links ]
    links = [l for l in links if l and 'all_comps/shooting' in l] #Get shooting links

    shooting_html = requests.get(f"https://fbref.com{links[0]}") #Convert to absolute url
    try:
      shooting_table = pd.read_html(shooting_html.text, match="Shooting")[0]
      shooting_table.head()
      shooting_table.columns = shooting_table.columns.droplevel()
    except ValueError:
      continue


    try:
      team_data = matches.merge(shooting_table[shooting_cols], on="Date") #Some shooting stats not available for all teams
    except ValueError:
      continue

    team_data[team_data["Comp"] == "Premier League"] #We are predicting premier league so for now we only want data in Premier League

    #Make sure you preserve important data
    team_data["Season"] = year
    team_data["Name"] = team_name

    all_matches.append(team_data)
    time.sleep(6.5) #Good practice so that you dont scrape too quickly and slow down website


ManchesterCity
Arsenal
Liverpool
AstonVilla
TottenhamHotspur
Chelsea
NewcastleUnited
ManchesterUnited
WestHamUnited
CrystalPalace
BrightonandHoveAlbion


ValueError: No tables found

match_df = pd.concat(all_matches)
all_matches

In [34]:
match_df = pd.concat(all_matches)
match_df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Sh,SoT,SoT%,G/Sh,G/SoT,FK,PK,xG_y,Season,Name
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (1),1 (4),Arsenal,...,8.0,4.0,50.0,0.13,0.25,NaN,0,NaN,2022,ManchesterCity
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,17.0,8.0,47.1,0.18,0.38,0.0,0,1.9,2022,ManchesterCity
2,2023-08-16,22:00,Super Cup,UEFA Super Cup,Wed,Home,D,1 (5),1 (4),es Sevilla,...,23.0,7.0,30.4,0.04,0.14,NaN,0,NaN,2022,ManchesterCity
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,14.0,4.0,28.6,0.07,0.25,0.0,0,1.0,2022,ManchesterCity
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,29.0,9.0,31.0,0.07,0.22,2.0,0,3.5,2022,ManchesterCity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2024-04-24,20:00,Premier League,Matchweek 29,Wed,Away,L,2,4,Manchester Utd,...,10.0,4.0,40.0,0.20,0.50,1.0,0,0.8,2021,SheffieldUnited
37,2024-04-27,15:00,Premier League,Matchweek 35,Sat,Away,L,1,5,Newcastle Utd,...,15.0,4.0,26.7,0.07,0.25,0.0,0,1.5,2021,SheffieldUnited
38,2024-05-04,15:00,Premier League,Matchweek 36,Sat,Home,L,1,3,Nott'ham Forest,...,16.0,4.0,25.0,0.00,0.00,0.0,1,2.2,2021,SheffieldUnited
39,2024-05-11,15:00,Premier League,Matchweek 37,Sat,Away,L,0,1,Everton,...,13.0,1.0,7.7,0.00,0.00,0.0,0,0.6,2021,SheffieldUnited


In [35]:
match_df.to_csv("matches.csv")